In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
import sklearn
import math
%matplotlib inline
import seaborn as sns
from collections import Counter, defaultdict

# Text Library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Splitting Data
from sklearn.model_selection import train_test_split

# Import libary for TFID Vectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import feature selection Libraries
from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_classif

# Algorithm
from sklearn.naive_bayes import MultinomialNB

# Evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dfTrain_PreStem = pd.read_csv('dfTrain_PreStem.csv')
dfTest_PreStem = pd.read_csv('dfTest_PreStem.csv')

fitur_prestem = pd.read_csv('fitur_prestem.csv')

In [3]:
fitur_prestem[fitur_prestem['feature'].isnull() == True]

,Unnamed: 0,feature,tfidf,ig_value
2592,2592,NaN,0.0,0.001751


In [4]:
fitur_prestem = fitur_prestem.drop(columns=['Unnamed: 0'])
fitur_prestem

,feature,tfidf,ig_value
0,aaaccchhh,0.0,0.000798
1,aaeu,0.0,0.000784
2,aamiin,0.0,0.001751
3,ab,0.0,0.000798
4,abad,0.0,0.000784
...,...,...,...
4416,zuudhvtnbc,0.0,0.000967
4417,zw,0.0,0.000784
4418,zww,0.0,0.000784
4419,zygzu,0.0,0.000967


In [5]:
fitur_prestem = fitur_prestem.dropna()
fitur_prestem

,feature,tfidf,ig_value
0,aaaccchhh,0.0,0.000798
1,aaeu,0.0,0.000784
2,aamiin,0.0,0.001751
3,ab,0.0,0.000798
4,abad,0.0,0.000784
...,...,...,...
4416,zuudhvtnbc,0.0,0.000967
4417,zw,0.0,0.000784
4418,zww,0.0,0.000784
4419,zygzu,0.0,0.000967


In [6]:
X_train_prestem = dfTrain_PreStem['tweet'].values
y_train_prestem = dfTrain_PreStem['label'].values

X_test_prestem = dfTest_PreStem['tweet'].values
y_test_prestem = dfTest_PreStem['label'].values

In [7]:
# Get feature train and test for dfPreStem
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

# Fit transform to data train
tfidf_train_prestem = tfidf_vectorizer.fit_transform(X_train_prestem)
tfidf_test_prestem = tfidf_vectorizer.transform(X_test_prestem)

# Get feature matrix for dfTrain_PreStem
fitur_train_prestem = tfidf_train_prestem.toarray()
fitur_tr4 = tfidf_vectorizer.get_feature_names()
train_prestem_mtx = pd.DataFrame(data=fitur_train_prestem,columns=fitur_tr4)

# Get feature matrix for dfTest_PreStem
fitur_test_prestem = tfidf_test_prestem.toarray()
fitur_te4 = tfidf_vectorizer.get_feature_names()
test_prestem_mtx = pd.DataFrame(data=fitur_test_prestem,columns=fitur_te4)

In [8]:
train_prestem_mtx['label_value'] = y_train_prestem

In [9]:
# Sort each fitur dataframe based on ig_value
fitur_prestem_sorted = fitur_prestem.sort_values(by='ig_value', ascending=False)
fitur_prestem_sorted = list(fitur_prestem_sorted.feature)

In [10]:
# Fungsi mengambil banyak top-k fitur yang harus diambil

def perc_fitur(train_vector):
    panjang = train_vector.shape[1]
    lenFitur = []
    for i in range (1,10):
      lenFitur.append(round((i/10)*panjang))
    lenFitur.reverse()
    
    return lenFitur

def generate_vec(train_data, test_data):
    X_train = train_data['tweet'].values
    y_train = train_data['label'].values

    X_test = test_data['tweet'].values
    y_test = test_data['label'].values
    
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

    # Fit transform to data train
    tfidf_train = tfidf_vectorizer.fit_transform(X_train)
    tfidf_test = tfidf_vectorizer.transform(X_test)
    
    return [tfidf_train, tfidf_test]

In [11]:
# Generate feature vector for each dataset
vec_prestem = generate_vec(dfTrain_PreStem, dfTest_PreStem)

In [12]:
# Ambil top-fitur setiap dataset dengan ukuran descending
lenFitur_prestem = perc_fitur(vec_prestem[0])

In [13]:
from sklearn import model_selection, svm

In [14]:
# Accuracy for df_pre1
acc_ig_prestem = []
for i in range(len(lenFitur_prestem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_prestem_sorted[:lenFitur_prestem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_prestem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_prestem_mtx[subset_feature].values)
    
#     # Build models and fit to train vector
#     mnb = MultinomialNB()
#     mnbTfidf = mnb.fit(features_train, y_train_prestem) # training the model
#     predictTfidf = mnbTfidf.predict(features_test)
        
#     # Append accuracy score for each 
#     acc_ig_pre1.append(accuracy_score(y_test_pre1,predictTfidf))
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(kernel = 'linear')
    SVM.fit(features_train, y_train_prestem)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVM.score(features_test, y_test_prestem)*100)

SVM Testing Accuracy Score ->  69.77611940298507
SVM Testing Accuracy Score ->  69.90049751243781
SVM Testing Accuracy Score ->  69.40298507462687
SVM Testing Accuracy Score ->  69.65174129353234
SVM Testing Accuracy Score ->  69.5273631840796
SVM Testing Accuracy Score ->  70.1492537313433
SVM Testing Accuracy Score ->  69.65174129353234
SVM Testing Accuracy Score ->  68.78109452736318
SVM Testing Accuracy Score ->  65.92039800995025


In [15]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline

In [16]:
lenFitur_prestem

[3979, 3537, 3095, 2653, 2210, 1768, 1326, 884, 442]

In [17]:
linear_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['linear']}
    
poly_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['poly'],
    'clf__degree' : [2, 3],
    'clf__gamma': np.logspace(-2, 2, 5)}

rbf_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['rbf'],
    'clf__gamma': np.logspace(-2, 2, 5)}

In [18]:
clfPreStem = svm.SVC()
tfidfPreStem = TfidfVectorizer()

pipePreStem = Pipeline([
    ('tfidf', tfidfPreStem),
    ('clf', clfPreStem)
])

In [19]:
scv_linear = RandomizedSearchCV(pipePreStem, param_distributions = linear_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_linear.fit(X_train_prestem, y_train_prestem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   31.6s finished


In [20]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_poly = RandomizedSearchCV(pipePreStem, param_distributions = poly_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_poly.fit(X_train_prestem, y_train_prestem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   34.6s finished


In [21]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_rbf = RandomizedSearchCV(pipePreStem, param_distributions = rbf_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_rbf.fit(X_train_prestem, y_train_prestem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   40.0s finished


In [22]:
#preview the best parameters as found by the gridsearch
scv_linear.best_params_

{'tfidf__ngram_range': (1, 1),
 'tfidf__max_features': None,
 'clf__kernel': 'linear',
 'clf__C': 10.0}

In [23]:
scv_poly.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'poly',
 'clf__gamma': 100.0,
 'clf__degree': 2,
 'clf__C': 100.0}

In [24]:
scv_rbf.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'rbf',
 'clf__gamma': 0.1,
 'clf__C': 100.0}

In [25]:
scv_rbf.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=0.1

In [26]:
lenFitur_prestem

[3979, 3537, 3095, 2653, 2210, 1768, 1326, 884, 442]

In [27]:
# Accuracy for df_pre1
acc_ig_prestem = []
for i in range(len(lenFitur_prestem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_prestem_sorted[:lenFitur_prestem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_prestem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_prestem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVML1 = svm.SVC(kernel = 'linear', C=10)
    SVML1.fit(features_train, y_train_prestem)

    # Use accuracy_score function to get the accuracy
    print("SVM Testing Accuracy Score -> ", SVML1.score(features_test, y_test_prestem)*100)

SVM Testing Accuracy Score ->  67.28855721393035
SVM Testing Accuracy Score ->  67.7860696517413
SVM Testing Accuracy Score ->  67.7860696517413
SVM Testing Accuracy Score ->  68.1592039800995
SVM Testing Accuracy Score ->  69.40298507462687
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  65.54726368159204
SVM Testing Accuracy Score ->  65.54726368159204
SVM Testing Accuracy Score ->  65.92039800995025


In [28]:
# Accuracy for df_pre1
acc_ig_prestem = []
for i in range(len(lenFitur_prestem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_prestem_sorted[:lenFitur_prestem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_prestem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_prestem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMP1 = svm.SVC(kernel = 'poly', degree = 2, gamma = 100, C = 100)
    SVMP1.fit(features_train, y_train_prestem)

    # Use accuracy_score function to get the accuracy
    print("SVM Testing Accuracy Score -> ", SVMP1.score(features_test, y_test_prestem)*100)

SVM Testing Accuracy Score ->  69.15422885572139
SVM Testing Accuracy Score ->  65.67164179104478
SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  67.16417910447761
SVM Testing Accuracy Score ->  69.40298507462687
SVM Testing Accuracy Score ->  63.557213930348254
SVM Testing Accuracy Score ->  62.56218905472637
SVM Testing Accuracy Score ->  56.840796019900495
SVM Testing Accuracy Score ->  61.19402985074627


In [29]:
# Accuracy for df_pre1
acc_ig_prestem = []
for i in range(len(lenFitur_prestem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_prestem_sorted[:lenFitur_prestem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_prestem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_prestem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMR1 = svm.SVC(kernel = 'rbf', gamma = 0.1, C = 100)
    SVMR1.fit(features_train, y_train_prestem)

    # Use accuracy_score function to get the accuracy
    print("SVM Testing Accuracy Score -> ", SVMR1.score(features_test, y_test_prestem)*100)

SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  66.66666666666666
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  66.7910447761194
SVM Testing Accuracy Score ->  66.16915422885572
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  65.29850746268657
SVM Testing Accuracy Score ->  65.04975124378109
SVM Testing Accuracy Score ->  66.7910447761194


In [30]:
lenFitur_prestem

[3979, 3537, 3095, 2653, 2210, 1768, 1326, 884, 442]

In [31]:
#obtaining predictions and decision function scores
Pred_YPreStem = SVML1.predict(features_test)

In [33]:
wrong_classified_PreStem = Pred_YPreStem != y_test_prestem

In [34]:
wrong_classified_PreStem

array([ True, False, False, False,  True, False, False, False, False,
        True, False, False,  True, False,  True, False, False,  True,
        True,  True, False, False,  True, False,  True, False, False,
       False,  True,  True,  True, False, False,  True,  True,  True,
       False,  True, False,  True, False, False, False, False, False,
        True, False,  True, False, False,  True, False,  True,  True,
       False,  True, False, False, False, False,  True, False, False,
       False, False,  True, False, False,  True,  True, False,  True,
        True,  True, False,  True,  True,  True, False, False,  True,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False, False,  True, False, False,  True, False, False, False,
        True, False, False,  True,  True,  True, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False,

In [35]:
wrong_data_PreStem = dfTest_PreStem[wrong_classified_PreStem == True]
wrong_labels_PreStem = Pred_YPreStem[wrong_classified_PreStem == True]

In [36]:
false_pronet_PreStem = wrong_data_PreStem[wrong_labels_PreStem == 1]
false_pronet_PreStem

,tweet,label
0,aspek historis calon ibu kota negara kuta kart...,0
4,negara kelola sporadis,0
17,pindah ibu kota trending twitter lihat koment...,0
18,proses pindah makan biaya t bagi tanggung wila...,-1
29,skema pembiayan pindah ibu kota negara lewat,0
...,...,...
727,jual negara halus pindah ibu kota dgn dominas...,-1
759,geopolitik pindah ibu kota negara sangat bahaya,-1
770,pindah ibu kota merta langsung perlu tuju...,0
793,amien rais perintah tunggu kaji tiongkok soal ...,-1


In [37]:
false_procon_PreStem = wrong_data_PreStem[wrong_labels_PreStem == 0]
false_procon_PreStem

,tweet,label
12,abis ngumumin pindah ibu kota negara biasa jaw...,-1
24,timbang tambah utang gitu maksud nikmat geli...,-1
52,apbn biaya pindah ibu kota kalimantan timur ...,1
53,masyarakat gunung mas minta hormat putus pinda...,1
55,mohon kaji kabid mitigasi gempa tsunami bmkg...,-1
...,...,...
725,yah skrng khan lg tren yg unfaedah kyk pindah ...,-1
730,benar pindah ibukota ringan beban jakrta yg ...,1
739,pindah ibu kota hak presiden pic twitter com ...,1
740,median rilis dapat masyarakat kait pindah ibu ...,-1


In [38]:
false_netcon_PreStem = wrong_data_PreStem[wrong_labels_PreStem == -1]
false_netcon_PreStem

,tweet,label
9,tuju alas realisasi sila adil sosial seluruh...,1
14,bapak tdk tolak tp bapak cuma jelas kakurangan...,0
19,kait rencana pindah ibu kota kalimantan banya...,1
22,ngebayangin gmna proses pindah ibu kota yg ...,0
28,bicara untung rugi pindah ibu kota negara lu...,0
...,...,...
767,amien rais bahas rencana pindah ibu kota,0
780,ppp nilai kritik ketua dewan hormat pan amien ...,1
783,soal pindah ibu kota kaltim amien rais keluar...,0
796,pindah ibu kota amien rais caper,0


# rawstem

In [39]:
dfTrain_RawStem = pd.read_csv('dfTrain_RawStem.csv')
dfTest_RawStem = pd.read_csv('dfTest_RawStem.csv')

fitur_rawstem = pd.read_csv('ig_rawstem.csv')

In [40]:
fitur_rawstem[fitur_rawstem['feature'].isnull() == True]

,Unnamed: 0,feature,tfidf,ig_value
2821,2821,NaN,0.0,0.001751


In [41]:
fitur_rawstem = fitur_rawstem.drop(columns=['Unnamed: 0'])
fitur_rawstem

,feature,tfidf,ig_value
0,00,0.0,0.006409
1,000,0.0,0.001569
2,03,0.0,0.000784
3,03persatuanindonesia,0.0,0.001582
4,05,0.0,0.002718
...,...,...,...
4689,zulkieflimansyah,0.0,0.000967
4690,zulkifli,0.0,0.000798
4691,zuudhvtnbc,0.0,0.000967
4692,zww4y9zufc,0.0,0.000784


In [42]:
fitur_rawstem = fitur_rawstem.dropna()
fitur_rawstem

,feature,tfidf,ig_value
0,00,0.0,0.006409
1,000,0.0,0.001569
2,03,0.0,0.000784
3,03persatuanindonesia,0.0,0.001582
4,05,0.0,0.002718
...,...,...,...
4689,zulkieflimansyah,0.0,0.000967
4690,zulkifli,0.0,0.000798
4691,zuudhvtnbc,0.0,0.000967
4692,zww4y9zufc,0.0,0.000784


In [43]:
X_train_rawstem = dfTrain_RawStem['tweet'].values
y_train_rawstem = dfTrain_RawStem['label'].values

X_test_rawstem = dfTest_RawStem['tweet'].values
y_test_rawstem = dfTest_RawStem['label'].values

In [44]:
# Get feature train and test for df_pre1
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

# Fit transform to data train
tfidf_train_rawstem = tfidf_vectorizer.fit_transform(X_train_rawstem)
tfidf_test_rawstem = tfidf_vectorizer.transform(X_test_rawstem)

# Get feature matrix for train_pre1
fitur_train_rawstem = tfidf_train_rawstem.toarray()
fitur_tr2 = tfidf_vectorizer.get_feature_names()
train_rawstem_mtx = pd.DataFrame(data=fitur_train_rawstem,columns=fitur_tr2)

# Get feature matrix for test_pre1
fitur_test_rawstem = tfidf_test_rawstem.toarray()
fitur_te2 = tfidf_vectorizer.get_feature_names()
test_rawstem_mtx = pd.DataFrame(data=fitur_test_rawstem,columns=fitur_te2)

In [45]:
train_rawstem_mtx['label_value'] = y_train_rawstem

In [46]:
# Sort each fitur dataframe based on ig_value
fitur_rawstem_sorted = fitur_rawstem.sort_values(by='ig_value', ascending=False)
fitur_rawstem_sorted = list(fitur_rawstem_sorted.feature)

In [47]:
# Fungsi mengambil banyak top-k fitur yang harus diambil

def perc_fitur(train_vector):
    panjang = train_vector.shape[1]
    lenFitur = []
    for i in range (1,10):
      lenFitur.append(round((i/10)*panjang))
    lenFitur.reverse()
    
    return lenFitur

def generate_vec(train_data, test_data):
    X_train = train_data['tweet'].values
    y_train = train_data['label'].values

    X_test = test_data['tweet'].values
    y_test = test_data['label'].values
    
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

    # Fit transform to data train
    tfidf_train = tfidf_vectorizer.fit_transform(X_train)
    tfidf_test = tfidf_vectorizer.transform(X_test)
    
    return [tfidf_train, tfidf_test]

In [48]:
# Generate feature vector for each dataset
vec_rawstem = generate_vec(dfTrain_RawStem, dfTest_RawStem)

In [49]:
# Ambil top-fitur setiap dataset dengan ukuran descending
lenFitur_rawstem = perc_fitur(vec_rawstem[0])

In [50]:
# Accuracy for dfRawStem
acc_ig_rawstem = []
for i in range(len(lenFitur_rawstem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_rawstem_sorted[:lenFitur_rawstem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_rawstem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_rawstem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(kernel = 'linear')
    SVM.fit(features_train, y_train_rawstem)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVM.score(features_test, y_test_rawstem)*100)

SVM Testing Accuracy Score ->  70.77114427860697
SVM Testing Accuracy Score ->  71.14427860696517
SVM Testing Accuracy Score ->  71.26865671641791
SVM Testing Accuracy Score ->  71.26865671641791
SVM Testing Accuracy Score ->  70.77114427860697
SVM Testing Accuracy Score ->  69.65174129353234
SVM Testing Accuracy Score ->  69.77611940298507
SVM Testing Accuracy Score ->  69.40298507462687
SVM Testing Accuracy Score ->  67.91044776119402


In [51]:
lenFitur_rawstem

[4225, 3755, 3286, 2816, 2347, 1878, 1408, 939, 469]

In [52]:
linear_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['linear']}
    
poly_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['poly'],
    'clf__degree' : [2, 3],
    'clf__gamma': np.logspace(-2, 2, 5)}

rbf_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['rbf'],
    'clf__gamma': np.logspace(-2, 2, 5)}

In [53]:
clfRawStem = svm.SVC()
tfidfRawStem = TfidfVectorizer()

pipeRawStem = Pipeline([
    ('tfidf', tfidfRawStem),
    ('clf', clfRawStem)
])

In [54]:
clfRawStem = svm.SVC()
tfidfRawStem = TfidfVectorizer()

pipeRawStem = Pipeline([
    ('tfidf', tfidfRawStem),
    ('clf', clfRawStem)
])

In [55]:
scv_linear = RandomizedSearchCV(pipeRawStem, param_distributions = linear_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_linear.fit(X_train_rawstem, y_train_rawstem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   37.9s finished


In [56]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_poly = RandomizedSearchCV(pipeRawStem, param_distributions = poly_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_poly.fit(X_train_rawstem, y_train_rawstem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   53.1s finished


In [57]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_rbf = RandomizedSearchCV(pipeRawStem, param_distributions = rbf_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_rbf.fit(X_train_rawstem, y_train_rawstem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   54.3s finished


In [58]:
#preview the best parameters as found by the gridsearch
scv_linear.best_params_

{'tfidf__ngram_range': (1, 1),
 'tfidf__max_features': None,
 'clf__kernel': 'linear',
 'clf__C': 10.0}

In [59]:
#preview the best parameters as found by the gridsearch
scv_poly.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': 5000,
 'clf__kernel': 'poly',
 'clf__gamma': 10.0,
 'clf__degree': 2,
 'clf__C': 10.0}

In [60]:
#preview the best parameters as found by the gridsearch
scv_rbf.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': 5000,
 'clf__kernel': 'rbf',
 'clf__gamma': 1.0,
 'clf__C': 10.0}

In [61]:
lenFitur_rawstem

[4225, 3755, 3286, 2816, 2347, 1878, 1408, 939, 469]

In [62]:
# Accuracy for dfRawStem
acc_ig_rawstem = []
for i in range(len(lenFitur_rawstem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_rawstem_sorted[:lenFitur_rawstem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_rawstem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_rawstem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVML2 = svm.SVC(kernel = 'linear', C= 10)
    SVML2.fit(features_train, y_train_rawstem)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVML2.score(features_test, y_test_rawstem)*100)

SVM Testing Accuracy Score ->  69.27860696517413
SVM Testing Accuracy Score ->  67.16417910447761
SVM Testing Accuracy Score ->  67.16417910447761
SVM Testing Accuracy Score ->  68.1592039800995
SVM Testing Accuracy Score ->  68.03482587064677
SVM Testing Accuracy Score ->  69.02985074626866
SVM Testing Accuracy Score ->  65.29850746268657
SVM Testing Accuracy Score ->  65.92039800995025
SVM Testing Accuracy Score ->  69.77611940298507


In [63]:
# Accuracy for dfRawStem
acc_ig_rawstem = []
for i in range(len(lenFitur_rawstem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_rawstem_sorted[:lenFitur_rawstem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_rawstem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_rawstem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMP2 = svm.SVC(kernel = 'poly', C= 10, degree = 2, gamma = 10)
    SVMP2.fit(features_train, y_train_rawstem)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMP2.score(features_test, y_test_rawstem)*100)

SVM Testing Accuracy Score ->  70.1492537313433
SVM Testing Accuracy Score ->  67.7860696517413
SVM Testing Accuracy Score ->  69.5273631840796
SVM Testing Accuracy Score ->  70.39800995024875
SVM Testing Accuracy Score ->  71.64179104477611
SVM Testing Accuracy Score ->  70.27363184079603
SVM Testing Accuracy Score ->  69.5273631840796
SVM Testing Accuracy Score ->  67.41293532338308
SVM Testing Accuracy Score ->  65.92039800995025


In [64]:
# Accuracy for dfRawStem
acc_ig_rawstem = []
for i in range(len(lenFitur_rawstem)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_rawstem_sorted[:lenFitur_rawstem[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_rawstem_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_rawstem_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMR2 = svm.SVC(kernel = 'poly', C= 10, gamma = 1)
    SVMR2.fit(features_train, y_train_rawstem)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMR2.score(features_test, y_test_rawstem)*100)

SVM Testing Accuracy Score ->  56.59203980099502
SVM Testing Accuracy Score ->  52.363184079601986
SVM Testing Accuracy Score ->  55.59701492537313
SVM Testing Accuracy Score ->  66.29353233830845
SVM Testing Accuracy Score ->  67.53731343283582
SVM Testing Accuracy Score ->  63.557213930348254
SVM Testing Accuracy Score ->  66.54228855721394
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  67.16417910447761


In [57]:
lenFitur_rawstem

[4225, 3755, 3286, 2816, 2347, 1878, 1408, 939, 469]

In [65]:
Pred_YRawStem = SVMP2.predict(features_test)

In [66]:
wrong_classified_RawStem = Pred_YRawStem != y_test_rawstem

In [67]:
wrong_classified_RawStem

array([ True, False, False,  True, False, False, False,  True,  True,
       False, False,  True,  True,  True,  True, False, False,  True,
        True, False,  True, False,  True, False,  True,  True, False,
       False, False, False,  True, False, False, False,  True,  True,
       False,  True, False, False,  True, False, False, False, False,
        True, False, False, False, False, False, False,  True,  True,
       False,  True, False, False, False,  True,  True, False,  True,
       False, False, False, False, False,  True,  True, False, False,
        True, False, False, False,  True, False,  True, False,  True,
       False, False, False, False,  True, False,  True,  True, False,
       False,  True, False, False, False,  True, False, False, False,
        True, False, False,  True, False, False,  True, False,  True,
       False, False, False, False, False, False,  True,  True,  True,
       False, False,  True, False, False,  True, False, False,  True,
       False, False,

In [68]:
wrong_data_RawStem = dfTest_RawStem[wrong_classified_RawStem == True]
wrong_labels_RawStem = Pred_YRawStem[wrong_classified_RawStem == True]

In [69]:
false_pronet_RawStem = wrong_data_RawStem[wrong_labels_RawStem == 1]
false_pronet_RawStem

,tweet,label
3,lagi pula pindah ibu kota ke kaltim kes jadi p...,-1
7,pindah ibu kota ke kalimantan timur bagaimana ...,0
11,tanya sederhana adalah jk anda turut komentar ...,0
14,bapak tdk tolak tp bapak cuma jelas kakurangan...,0
20,pindah ibu kota baik lalu undang,-1
...,...,...
788,mendagri soal wacana pindah ibu kota jabar sia...,0
789,amein rais minta jokowi batal pindah ibu kota ...,-1
791,pindah ibu kota ke kaltim akan turun aktivitas...,-1
793,amien rais perintah tunggu kaji tiongkok soal ...,-1


In [70]:
false_procon_RawStem = wrong_data_RawStem[wrong_labels_RawStem == 0]
false_procon_RawStem

,tweet,label
12,abis ngumumin pindah ibu kota negara biasa jaw...,-1
18,proses pindah makan biaya 466 t bagi tanggung ...,-1
24,timbang untuk tambah utang gitu maksud dan nik...,-1
35,pak dar tanya yah apakah ada yang jamin kalo p...,-1
53,masyarakat gunung mas minta hormat putus pinda...,1
...,...,...
756,pindah ibu kota pasti butuh dana dong pak dana...,-1
758,fadli zon pindah ibu kota kes dadak dan gesa ...,-1
759,cara geopolitik pindah ibu kota negara sangat ...,-1
771,fadli zon yakin pindah ibu kota di tahun 2024 ...,-1


In [71]:
false_netcon_RawStem = wrong_data_RawStem[wrong_labels_RawStem == -1]
false_netcon_RawStem

,tweet,label
0,aspek historis calon ibu kota negara kuta kart...,0
8,ada beberapa aspek yang kaji dalam pindah ibu ...,0
13,kakak dapid td lg meeting pindah ibu kota kata,0
17,pindah ibu kota trending di twitter lihat kome...,0
22,masih ngebayangin gmna proses pindah ibu kota ...,0
...,...,...
770,pindah ibu kota ini tidak serta merta langsung...,0
773,bukankah pindah ibu kota adalah cita2 bung kar...,1
796,pindah ibu kota amien rais caper pdiperjuangan,0
800,pindah ibu kota dengan skema investasi dari ju...,1


# Raw

In [72]:
dfTrain_Raw = pd.read_csv('dfTrain_Raw.csv')
dfTest_Raw = pd.read_csv('dfTest_Raw.csv')

fitur_raw = pd.read_csv('ig_raw.csv')

In [73]:
fitur_raw[fitur_raw['feature'].isnull() == True]

,Unnamed: 0,feature,tfidf,ig_value
5864,5864,NaN,0.0,0.001751


In [74]:
fitur_raw = fitur_raw.drop(columns=['Unnamed: 0'])
fitur_raw

,feature,tfidf,ig_value
0,00,0.0,0.006409
1,000,0.0,0.001569
2,03,0.0,0.000784
3,03PersatuanIndonesia,0.0,0.001582
4,05,0.0,0.002718
...,...,...,...
7559,zul,0.0,0.002366
7560,اطلبوا,0.0,0.000784
7561,العلم,0.0,0.000784
7562,بالصين,0.0,0.000784


In [75]:
fitur_raw = fitur_raw.dropna()
fitur_raw

,feature,tfidf,ig_value
0,00,0.0,0.006409
1,000,0.0,0.001569
2,03,0.0,0.000784
3,03PersatuanIndonesia,0.0,0.001582
4,05,0.0,0.002718
...,...,...,...
7559,zul,0.0,0.002366
7560,اطلبوا,0.0,0.000784
7561,العلم,0.0,0.000784
7562,بالصين,0.0,0.000784


In [76]:
X_train_raw = dfTrain_Raw['tweet'].values
y_train_raw = dfTrain_Raw['label'].values

X_test_raw = dfTest_Raw['tweet'].values
y_test_raw = dfTest_Raw['label'].values

In [77]:
# Get feature train and test for dfRaw
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

# Fit transform to data train
tfidf_train_raw = tfidf_vectorizer.fit_transform(X_train_raw)
tfidf_test_raw = tfidf_vectorizer.transform(X_test_raw)

# Get feature matrix for dfTrain_Raw
fitur_train_raw = tfidf_train_raw.toarray()
fitur_tr3 = tfidf_vectorizer.get_feature_names()
train_raw_mtx = pd.DataFrame(data=fitur_train_raw,columns=fitur_tr3)

# Get feature matrix for dfTest_Raw
fitur_test_raw = tfidf_test_raw.toarray()
fitur_te3 = tfidf_vectorizer.get_feature_names()
test_raw_mtx = pd.DataFrame(data=fitur_test_raw,columns=fitur_te3)

In [78]:
train_raw_mtx['label_value'] = y_train_raw

In [79]:
# Sort each fitur dataframe based on ig_value
fitur_raw_sorted = fitur_raw.sort_values(by='ig_value', ascending=False)
fitur_raw_sorted = list(fitur_raw_sorted.feature)

In [80]:
# Fungsi mengambil banyak top-k fitur yang harus diambil

def perc_fitur(train_vector):
    panjang = train_vector.shape[1]
    lenFitur = []
    for i in range (1,10):
      lenFitur.append(round((i/10)*panjang))
    lenFitur.reverse()
    
    return lenFitur

def generate_vec(train_data, test_data):
    X_train = train_data['tweet'].values
    y_train = train_data['label'].values

    X_test = test_data['tweet'].values
    y_test = test_data['label'].values
    
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

    # Fit transform to data train
    tfidf_train = tfidf_vectorizer.fit_transform(X_train)
    tfidf_test = tfidf_vectorizer.transform(X_test)
    
    return [tfidf_train, tfidf_test]

In [81]:
# Generate feature vector for each dataset
vec_raw = generate_vec(dfTrain_Raw, dfTest_Raw)

In [82]:
# Ambil top-fitur setiap dataset dengan ukuran descending
lenFitur_raw = perc_fitur(vec_raw[0])

In [83]:
# Accuracy for dfRawStem
acc_ig_raw = []
for i in range(len(lenFitur_raw)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_raw_sorted[:lenFitur_raw[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_raw_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_raw_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(kernel = 'linear')
    SVM.fit(features_train, y_train_raw)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVM.score(features_test, y_test_raw)*100)

SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  68.53233830845771
SVM Testing Accuracy Score ->  67.7860696517413
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  66.54228855721394
SVM Testing Accuracy Score ->  65.4228855721393
SVM Testing Accuracy Score ->  64.80099502487562
SVM Testing Accuracy Score ->  62.68656716417911


In [84]:
lenFitur_raw

[6808, 6051, 5295, 4538, 3782, 3026, 2269, 1513, 756]

In [85]:
linear_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['linear']}
    
poly_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['poly'],
    'clf__degree' : [2, 3],
    'clf__gamma': np.logspace(-2, 2, 5)}

rbf_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['rbf'],
    'clf__gamma': np.logspace(-2, 2, 5)}

In [86]:
clfRaw = svm.SVC()
tfidfRaw = TfidfVectorizer()

pipeRaw = Pipeline([
    ('tfidf', tfidfRaw),
    ('clf', clfRaw)
])

In [87]:
scv_linear = RandomizedSearchCV(pipeRaw, param_distributions = linear_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_linear.fit(X_train_raw, y_train_raw);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   31.7s finished


In [88]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_poly = RandomizedSearchCV(pipeRaw, param_distributions = poly_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_poly.fit(X_train_raw, y_train_raw);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   47.9s finished


In [89]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_rbf = RandomizedSearchCV(pipeRaw, param_distributions = rbf_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_rbf.fit(X_train_raw, y_train_rawstem);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   51.5s finished


In [90]:
#preview the best parameters as found by the gridsearch
scv_linear.best_params_

{'tfidf__ngram_range': (1, 1),
 'tfidf__max_features': None,
 'clf__kernel': 'linear',
 'clf__C': 10.0}

In [91]:
#preview the best parameters as found by the gridsearch
scv_poly.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'poly',
 'clf__gamma': 100.0,
 'clf__degree': 2,
 'clf__C': 100.0}

In [92]:
#preview the best parameters as found by the gridsearch
scv_rbf.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'rbf',
 'clf__gamma': 0.1,
 'clf__C': 100.0}

In [93]:
lenFitur_raw

[6808, 6051, 5295, 4538, 3782, 3026, 2269, 1513, 756]

In [95]:
# Accuracy for dfRawStem
acc_ig_raw = []
for i in range(len(lenFitur_raw)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_raw_sorted[:lenFitur_raw[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_raw_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_raw_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVML3 = svm.SVC(kernel = 'linear', C= 10)
    SVML3.fit(features_train, y_train_raw)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVML3.score(features_test, y_test_raw)*100)

SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  68.03482587064677
SVM Testing Accuracy Score ->  68.03482587064677
SVM Testing Accuracy Score ->  66.91542288557214
SVM Testing Accuracy Score ->  66.4179104477612
SVM Testing Accuracy Score ->  64.80099502487562
SVM Testing Accuracy Score ->  61.69154228855721


In [96]:
# Accuracy for dfRawStem
acc_ig_raw = []
for i in range(len(lenFitur_raw)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_raw_sorted[:lenFitur_raw[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_raw_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_raw_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMP3 = svm.SVC(kernel = 'poly', C= 100, gamma = 100, degree = 2)
    SVMP3.fit(features_train, y_train_raw)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMP3.score(features_test, y_test_raw)*100)

SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  59.950248756218905
SVM Testing Accuracy Score ->  63.43283582089553
SVM Testing Accuracy Score ->  65.67164179104478
SVM Testing Accuracy Score ->  65.54726368159204
SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  61.318407960199
SVM Testing Accuracy Score ->  52.736318407960205
SVM Testing Accuracy Score ->  59.5771144278607


In [97]:
# Accuracy for dfRawStem
acc_ig_raw = []
for i in range(len(lenFitur_raw)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_raw_sorted[:lenFitur_raw[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_raw_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_raw_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMR3 = svm.SVC(kernel = 'rbf', C= 100, gamma = 0.1)
    SVMR3.fit(features_train, y_train_raw)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMR3.score(features_test, y_test_raw)*100)

SVM Testing Accuracy Score ->  69.02985074626866
SVM Testing Accuracy Score ->  68.1592039800995
SVM Testing Accuracy Score ->  67.91044776119402
SVM Testing Accuracy Score ->  67.16417910447761
SVM Testing Accuracy Score ->  66.29353233830845
SVM Testing Accuracy Score ->  67.28855721393035
SVM Testing Accuracy Score ->  66.29353233830845
SVM Testing Accuracy Score ->  65.4228855721393
SVM Testing Accuracy Score ->  63.557213930348254


In [98]:
lenFitur_raw

[6808, 6051, 5295, 4538, 3782, 3026, 2269, 1513, 756]

In [99]:
Pred_YRaw = SVMR3.predict(features_test)

In [100]:
wrong_classified_Raw = Pred_YRaw != y_test_raw

In [101]:
wrong_classified_Raw

array([ True, False, False, False, False, False,  True,  True, False,
       False, False,  True, False,  True,  True, False, False,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True,  True, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False,  True, False,  True,
        True, False, False, False, False,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True, False, False,
       False,  True,  True, False, False,  True, False, False, False,
        True,  True, False,  True, False,  True, False, False,  True,
       False, False, False, False, False, False,  True, False,  True,
       False,  True, False, False, False,  True,  True, False,  True,
       False, False,

In [102]:
wrong_data_Raw = dfTest_Raw[wrong_classified_Raw == True]
wrong_labels_Raw = Pred_YRaw[wrong_classified_Raw == True]

In [104]:
false_pronet_Raw = wrong_data_Raw[wrong_labels_Raw == 1]
false_pronet_Raw

,tweet,label
0,Aspek historis calon ibu kota negara Kutai Ka...,0
6,Mudah2an biaya pemindahan ibu kota tidak menja...,-1
7,Pemindahan ibu kota ke Kalimantan Timur bagaim...,0
17,"Pemindahan Ibu Kota Trending di Twitter, Lihat...",0
23,F-PDIP: Pemindahan Ibu Kota Pernah Dibicarakan...,0
...,...,...
738,Dualisme Konseptual antara Perbaikan atau Pemi...,0
740,Median merilis pendapat masyarakat terkait pem...,-1
741,Gegap gempita soal pemindahan ibu kota...gilir...,-1
759,"Secara Geopolitik, Pemindahan Ibu Kota Negara ...",-1


In [105]:
false_procon_Raw = wrong_data_Raw[wrong_labels_Raw == 0]
false_procon_Raw

,tweet,label
18,"proses pemindahan memakan biaya 466 T, sebagia...",-1
20,Pemindahan Ibu Kota Sebaiknya Melalui Undang-u...,-1
24,Dipetimbangkan untuk menambah utang gitu maksu...,-1
31,"Bisnis Ekspedisi Bakal Meningkat, Seiring Pemi...",1
35,Pak sekedar tanya yah apakah ada yang menjamin...,-1
...,...,...
723,Bumi Resources (BUMI) melihat potensi untung d...,1
775,"≈. ≈. Soal Pemindahan Ibu Kota, Amien Rais: Ba...",-1
777,"//. Soal Pemindahan Ibu Kota, Amien Rais: Bata...",-1
791,Pemindahan ibu kota ke Kaltim akan turunkan ak...,-1


In [106]:
false_netcon_Raw = wrong_data_Raw[wrong_labels_Raw == -1]
false_netcon_Raw

,tweet,label
11,"Pertanyaan sederhananya adalah, jk Anda turut ...",0
13,Kakak dapid td lg meeting pemindahan ibu kota ...,0
14,bapak tdk menolak tp bapak cuma menjelaskan ka...,0
19,Terkait rencana pemindahan Ibu Kota di kaliman...,1
22,Masih ngebayangin gmna proses pemindahan ibu k...,0
...,...,...
780,PPP menilai kritik Ketua Dewan Kehormatan PAN ...,1
796,"Pemindahan Ibu kota, Amien Rais Caper! #pdip...",0
797,Pemindahan Ibu Kota RI dari Perspektif Pertaha...,0
799,Fadli Zon Pertanyakan Urgensi Pemindahan Ibu K...,0


# Pre

In [107]:
dfTrain_Pre = pd.read_csv('dfTrain_Pre.csv')
dfTest_Pre = pd.read_csv('dfTest_Pre.csv')

fitur_pre = pd.read_csv('ig_pre.csv')

In [108]:
fitur_pre[fitur_pre['feature'].isnull() == True]

,Unnamed: 0,feature,tfidf,ig_value
3532,3532,NaN,0.0,0.001751


In [109]:
fitur_pre = fitur_pre.drop(columns=['Unnamed: 0'])
fitur_pre

,feature,tfidf,ig_value
0,aaaccchhh,0.0,0.000798
1,aaeu,0.0,0.000784
2,aamiin,0.0,0.001751
3,ab,0.0,0.000798
4,abad,0.0,0.000784
...,...,...,...
5794,zuudhvtnbc,0.0,0.000967
5795,zw,0.0,0.000784
5796,zww,0.0,0.000784
5797,zygzu,0.0,0.000967


In [110]:
fitur_pre = fitur_pre.dropna()
fitur_pre

,feature,tfidf,ig_value
0,aaaccchhh,0.0,0.000798
1,aaeu,0.0,0.000784
2,aamiin,0.0,0.001751
3,ab,0.0,0.000798
4,abad,0.0,0.000784
...,...,...,...
5794,zuudhvtnbc,0.0,0.000967
5795,zw,0.0,0.000784
5796,zww,0.0,0.000784
5797,zygzu,0.0,0.000967


In [111]:
X_train_pre = dfTrain_Pre['tweet'].values
y_train_pre = dfTrain_Pre['label'].values

X_test_pre = dfTest_Pre['tweet'].values
y_test_pre = dfTest_Pre['label'].values

In [112]:
# Get feature train and test for dfPre
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

# Fit transform to data train
tfidf_train_pre = tfidf_vectorizer.fit_transform(X_train_pre)
tfidf_test_pre = tfidf_vectorizer.transform(X_test_pre)

# Get feature matrix for dfTrain_Pre
fitur_train_pre = tfidf_train_pre.toarray()
fitur_tr1 = tfidf_vectorizer.get_feature_names()
train_pre_mtx = pd.DataFrame(data=fitur_train_pre,columns=fitur_tr1)

# Get feature matrix for dfTest_Pre
fitur_test_pre = tfidf_test_pre.toarray()
fitur_te1 = tfidf_vectorizer.get_feature_names()
test_pre_mtx = pd.DataFrame(data=fitur_test_pre,columns=fitur_te1)

In [113]:
train_pre_mtx['label_value'] = y_train_pre

In [114]:
# Sort each fitur dataframe based on ig_value
fitur_pre_sorted = fitur_pre.sort_values(by='ig_value', ascending=False)
fitur_pre_sorted = list(fitur_pre_sorted.feature)

In [115]:
# Fungsi mengambil banyak top-k fitur yang harus diambil

def perc_fitur(train_vector):
    panjang = train_vector.shape[1]
    lenFitur = []
    for i in range (1,10):
      lenFitur.append(round((i/10)*panjang))
    lenFitur.reverse()
    
    return lenFitur

def generate_vec(train_data, test_data):
    X_train = train_data['tweet'].values
    y_train = train_data['label'].values

    X_test = test_data['tweet'].values
    y_test = test_data['label'].values
    
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

    # Fit transform to data train
    tfidf_train = tfidf_vectorizer.fit_transform(X_train)
    tfidf_test = tfidf_vectorizer.transform(X_test)
    
    return [tfidf_train, tfidf_test]

In [116]:
# Generate feature vector for each dataset
vec_pre = generate_vec(dfTrain_Pre, dfTest_Pre)

In [117]:
# Ambil top-fitur setiap dataset dengan ukuran descending
lenFitur_pre = perc_fitur(vec_pre[0])

In [118]:
# Accuracy for df_pre1
acc_ig_pre = []
for i in range(len(lenFitur_pre)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_pre_sorted[:lenFitur_pre[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_pre_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_pre_mtx[subset_feature].values)
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(kernel = 'linear')
    SVM.fit(features_train, y_train_pre)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVM.score(features_test, y_test_pre)*100)

SVM Testing Accuracy Score ->  70.52238805970148
SVM Testing Accuracy Score ->  70.27363184079603
SVM Testing Accuracy Score ->  70.1492537313433
SVM Testing Accuracy Score ->  70.52238805970148
SVM Testing Accuracy Score ->  69.77611940298507
SVM Testing Accuracy Score ->  69.27860696517413
SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  64.80099502487562


In [119]:
lenFitur_pre

[5219, 4639, 4059, 3479, 2900, 2320, 1740, 1160, 580]

In [120]:
linear_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['linear']}
    
poly_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['poly'],
    'clf__degree' : [2, 3],
    'clf__gamma': np.logspace(-2, 2, 5)}

rbf_grid = {'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': np.logspace(-1, 3, 5),
    'clf__kernel' : ['rbf'],
    'clf__gamma': np.logspace(-2, 2, 5)}

In [121]:
clfPre = svm.SVC()
tfidfPre = TfidfVectorizer()

pipePre = Pipeline([
    ('tfidf', tfidfPre),
    ('clf', clfPre)
])

In [122]:
scv_linear = RandomizedSearchCV(pipePre, param_distributions = linear_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_linear.fit(X_train_pre, y_train_pre);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   27.3s finished


In [123]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_poly = RandomizedSearchCV(pipePre, param_distributions = poly_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_poly.fit(X_train_pre, y_train_pre);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   34.8s finished


In [124]:
#using a randomized grid search to find the optimal value of the hyperparameters over a 5-fold cross-validation
scv_rbf = RandomizedSearchCV(pipePre, param_distributions = rbf_grid, n_iter = 10, cv = 5, verbose = 2, n_jobs = -1, return_train_score = True, random_state = 42)
scv_rbf.fit(X_train_pre, y_train_pre);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   39.0s finished


In [125]:
#preview the best parameters as found by the gridsearch
scv_linear.best_params_

{'tfidf__ngram_range': (1, 1),
 'tfidf__max_features': None,
 'clf__kernel': 'linear',
 'clf__C': 10.0}

In [126]:
scv_poly.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'poly',
 'clf__gamma': 100.0,
 'clf__degree': 2,
 'clf__C': 100.0}

In [127]:
scv_rbf.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_features': None,
 'clf__kernel': 'rbf',
 'clf__gamma': 0.1,
 'clf__C': 100.0}

In [128]:
lenFitur_pre

[5219, 4639, 4059, 3479, 2900, 2320, 1740, 1160, 580]

In [129]:
# Accuracy for df_pre1
acc_ig_pre = []
for i in range(len(lenFitur_pre)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_pre_sorted[:lenFitur_pre[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_pre_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_pre_mtx[subset_feature].values)
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVML4 = svm.SVC(kernel = 'linear', C = 10)
    SVML4.fit(features_train, y_train_pre)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVML4.score(features_test, y_test_pre)*100)

SVM Testing Accuracy Score ->  68.28358208955224
SVM Testing Accuracy Score ->  68.40796019900498
SVM Testing Accuracy Score ->  68.03482587064677
SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  67.41293532338308
SVM Testing Accuracy Score ->  68.03482587064677
SVM Testing Accuracy Score ->  66.7910447761194
SVM Testing Accuracy Score ->  64.92537313432835
SVM Testing Accuracy Score ->  65.17412935323384


In [130]:
# Accuracy for df_pre1
acc_ig_pre = []
for i in range(len(lenFitur_pre)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_pre_sorted[:lenFitur_pre[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_pre_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_pre_mtx[subset_feature].values)
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMP4 = svm.SVC(kernel = 'poly', C = 100, gamma = 100, degree = 2)
    SVMP4.fit(features_train, y_train_pre)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMP4.score(features_test, y_test_pre)*100)

SVM Testing Accuracy Score ->  67.66169154228857
SVM Testing Accuracy Score ->  63.18407960199005
SVM Testing Accuracy Score ->  64.6766169154229
SVM Testing Accuracy Score ->  67.28855721393035
SVM Testing Accuracy Score ->  67.7860696517413
SVM Testing Accuracy Score ->  62.56218905472637
SVM Testing Accuracy Score ->  54.975124378109456
SVM Testing Accuracy Score ->  55.348258706467654
SVM Testing Accuracy Score ->  53.85572139303483


In [131]:
# Accuracy for df_pre1
acc_ig_pre = []
for i in range(len(lenFitur_pre)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_pre_sorted[:lenFitur_pre[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_pre_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_pre_mtx[subset_feature].values)
    
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVMR4 = svm.SVC(kernel = 'rbf', C = 100, gamma = 0.1)
    SVMR4.fit(features_train, y_train_pre)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVMR4.score(features_test, y_test_pre)*100)

SVM Testing Accuracy Score ->  68.1592039800995
SVM Testing Accuracy Score ->  66.54228855721394
SVM Testing Accuracy Score ->  65.4228855721393
SVM Testing Accuracy Score ->  64.92537313432835
SVM Testing Accuracy Score ->  64.80099502487562
SVM Testing Accuracy Score ->  66.7910447761194
SVM Testing Accuracy Score ->  65.29850746268657
SVM Testing Accuracy Score ->  64.55223880597015
SVM Testing Accuracy Score ->  63.681592039801


In [109]:
lenFitur_pre

[5219, 4639, 4059, 3479, 2900, 2320, 1740, 1160, 580]

In [132]:
Pred_YPre = SVML4.predict(features_test)

In [133]:
wrong_classified_Pre = Pred_YPre != y_test_pre
wrong_classified_Pre

array([ True, False, False, False,  True, False,  True, False, False,
       False, False, False, False,  True,  True, False, False,  True,
        True,  True, False, False, False, False,  True, False, False,
       False, False,  True,  True, False,  True,  True,  True, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True, False, False, False, False,
       False, False,  True, False, False, False,  True, False,  True,
        True,  True,  True, False,  True, False,  True, False,  True,
        True,  True, False, False,  True, False,  True, False, False,
       False, False,  True,  True, False,  True, False, False, False,
        True,  True, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True,  True, False, False,
        True, False, False,  True, False, False,  True, False,  True,
       False, False,

In [134]:
wrong_data_Pre = dfTest_Pre[wrong_classified_Pre == True]
wrong_labels_Pre = Pred_YPre[wrong_classified_Pre == True]

In [135]:
false_pronet_Pre = wrong_data_Pre[wrong_labels_Pre == 1]
false_pronet_Pre

,tweet,label
0,aspek historis calon ibu kota negara kutai kar...,0
4,negara dikelola sporadis,0
17,pemindahan ibu kota trending twitter lihat ko...,0
29,skema pembiayan pemindahan ibu kota negara lew...,0
30,pemindahan ibu kota pindah thn pemilihan presi...,0
...,...,...
715,sebut kondisi bangsa menjerit jalih pitoeng mi...,-1
727,menjual negara halus pemindahan ibu kota dgn ...,-1
759,geopolitik pemindahan ibu kota negara sangat ...,-1
783,soal pemindahan ibu kota kaltim amien rais ke...,0


In [136]:
false_procon_Pre = wrong_data_Pre[wrong_labels_Pre == 0]
false_procon_Pre

,tweet,label
6,mudah an biaya pemindahan ibu kota menjadi aj...,-1
18,proses pemindahan memakan biaya t sebagian dit...,-1
24,dipetimbangkan menambah utang gitu maksudnya ...,-1
32,sepertinya besar sengaja ditutupi dibalik ...,-1
48,kondisi plus hutang negara t berbanggalah ...,-1
...,...,...
739,pemindahan ibu kota hak presiden pic twitter ...,1
740,median merilis pendapat masyarakat terkait pem...,-1
752,masa bodo isu pemindahan ibu kota sing pentin...,-1
771,fadli zon yakini pemindahan ibu kota tahun sulit,-1


In [137]:
false_netcon_Pre = wrong_data_Pre[wrong_labels_Pre == -1]
false_netcon_Pre

,tweet,label
13,kakak dapid td lg meeting pemindahan ibu kota ...,0
14,bapak tdk menolak tp bapak cuma menjelaskan ka...,0
19,terkait rencana pemindahan ibu kota kalimanta...,1
33,terpecah belah cuma gara gara kepentingan poli...,0
34,kepikiran klo pemindahan ibu kota bukan pemiki...,0
...,...,...
770,pemindahan ibu kota merta berlangsung per...,0
780,ppp menilai kritik ketua dewan kehormatan pan ...,1
796,pemindahan ibu kota amien rais caper,0
799,fadli zon pertanyakan urgensi pemindahan ibu k...,0
